# Generate synthetic question and answers pairs from a document

This code demonstrate how to create questions and answers based on a document.

The output is a list of questions and answers from a document.

## Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
+ An Azure OpenAI service with the service name and an API key.
+ A deployment of the text-embedding-ada-002 embedding model on the Azure OpenAI Service.
+ An Azure AI Search service with the end-point, API Key and the index name to create.

We used Python 3.12.3, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install openai

## Import packages and create AOAI client

In [2]:
import os
from dotenv import load_dotenv
import json
from openai import AzureOpenAI
import sys
sys.path.append('..')
from pa_utils import load_files

# Load environment variables from .env
load_dotenv(override=True)

# AOAI FOR QUESTION&ANSWER PAIRS GENERATION
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_apikey = os.environ["AZURE_OPENAI_API_KEY"]
aoai_model_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
# GPT-3.5T
#aoai_endpoint = os.environ["AZURE_OPENAI_RERANK_ENDPOINT"]
#aoai_apikey = os.environ["AZURE_OPENAI_RERANK_API_KEY"]
#aoai_model_name = os.environ["AZURE_OPENAI_RERANK_DEPLOYMENT_NAME"]

# Create AOAI client for answer Q&A pairs generation
aoai_api_version = '2024-02-15-preview'
aoai_client = AzureOpenAI(
    azure_deployment=aoai_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint,
    api_key=aoai_apikey
)

## Generate synthetic questions and answers from a document

In [12]:
def generate_answers_and_questions(text):

    system_prompt = """
    Your mission is to generate questions and answers targeting the context provided with the following format:
    "question": "text of the question", "answer": "text of the answer"
    The answers and questions pairs has to be in the same language than the Context.
    Here you have some examples of context and the question and answer pairs as your response in json format:
    Context: Sarah found a lost kitten on the street and decided to take it home.
    Response:
    [
        {"question": "why did Sarah decide to take the kitten home?",
         "answer": "the kitten was lost"},
        {"question": "where did Sarah find the kitten?", 
         "answer": "on the street"},
    ]
    Context: Jack saw a friendly group of kids playing in the park, so he decided to join them.
    Response:
    [
        {"question": "why did Jack decide to join the group of kids playing in the park?", 
         "answer": "the group of kids was friendly"},
        {"question": "what were the group of kids doing?", 
         "answer": "playing in the park"}
    ]
    """

    user_prompt = f'Context: "{text}". Response: '
   
    response = call_aoai(aoai_client, aoai_model_name, system_prompt, user_prompt, 0.5, 4096)
    # print(f'RESPONSE: [{response}]')
    qa_list = []
    if response != None:
        qa_pairs = json.loads(response)
        i=1
        for qa_pair in qa_pairs:     
            print(f'\t[{i}]: question: [{qa_pair["question"]}]')
            print(f'\t answer: [{qa_pair["answer"]}]')
            qa_list.append(qa_pair)
            i+=1

    return qa_list

In [28]:
# Test

markdown = """
Descripci\u00f3n\n===\n\nEste servicio se cerrar\u00e1 comercialmente el 31 de marzo de 2011\n\n||\n| - |\n| Definici\u00f3n |\n| Servicio de suscripci\u00f3n que permite al cliente recibir mensajes en el m\u00f3vil con informaci\u00f3n sobre los correos recibidos en las cuentas de correo que haya configurado La v\u00eda de notificaci\u00f3n por defecto es MMS, pero se puede adaptar a las capacidades de su terminal y realizar el env\u00edo como wap push o SMS Existen dos servicios 1 Mensaje Mail Express Por cada correo que el cliente reciba en su buz\u00f3n, se enviar\u00e1 un mensaje con el contenido de dicho correo - La informaci\u00f3n que recibir\u00e1 incluye el remitente, asunto, mensaje original hasta 3 000 caracteres, y la URL para acceder al mensaje por emocion en caso de que fuera necesario ampliar la informaci\u00f3n No incluye im\u00e1genes - Este servicio est\u00e1 disponible para las cuentas de Windows Live Hotmail, Yahoo, Gmail, Terra, Telefonica net y todas aquellas que tengan habilitado el acceso POP 3/Imap 4 (ej Wanadoo, ?) - Para evitar la recepci\u00f3n de Mensajes Mail Express con contenido que el cliente no desea, se permite al cliente filtrar la recepci\u00f3n por: remitente, urgencia, hora de recepci\u00f3n e incluso d\u00eda - El precio del servicio solo incluye los correos recibidos Si el cliente desear\u00e1 responder a un correo recibido se tarificar\u00e1 a los precios regulares del MMS - El precio del servicio es 2 ? Impuestos Indirectos No Incluidos 2 Mensaje resumen Se produce el env\u00edo una vez al d\u00eda con los correos electr\u00f3nicos recibidos el d\u00eda anterior en la cuenta o cuentas que tenga configurada/s - Disponible para Hotmail, Yahoo, Terra, Telefonica net, movistar, y muchas m\u00e1s (cuentas que permitan un acceso POP3/IMAP4 o la activaci\u00f3n de un reenv\u00edo del correo, como Gmail) La informaci\u00f3n incluye el n? total de mensajes recibidos, remitente, asunto y URL de la direcci\u00f3n de emocion donde el cliente puede consultar el correo - El precio de suscripci\u00f3n a este servicio es de 1 euro al mes, impuestos indirectos no incluidos - En cliente puede planificar sus alertas resumen, de manera que pueda determinar cuando quiere recibirlas: - Ma\u00f1ana (09:00h-10 00h), - Tarde (15:00h-16 00h) - Noche (21:00h-22 00h) |\n| |\n| Publico Objetivo |\n| |\n| Internautas usuarios habituales de correo electr\u00f3nico que no deseen pagar un precio elevado por tener el correo en su m\u00f3vil Segmento Residencial , principalmente de edad comprendida entre los 25 y 35 a\u00f1os |\n\n||\n| - |\n| Condiciones/Funcionamiento |\n| |\n| Activacion y configuraci\u00f3n de cuenta |\n| |\n| Se habilitan distintos interfaces para la configuraci\u00f3n y activaci\u00f3n de la cuenta de correo Desde emocion Hay que entrar en el men\u00fa wap de e-moci\u00f3n( wap movistar es) A continuaci\u00f3n el usuario entrar\u00e1 en el enlace correspondiente a \"Otras cuentas de correo\" Ser\u00e1 necesario que el cliente introduzca los datos b\u00e1sicos de la cuenta de correo que quiere dar de alta Mail Express o Mensaje Resumen, usuario y contrase\u00f1a (la cuenta deber\u00e1 permitir POP3/IMAP4, en caso contrario (por ej, gmail) el sistema le pedir\u00e1 que active el reenvio autom\u00e1tico en el servidor Una vez ?movilizada? la cuenta tendr\u00e1 la opci\u00f3n de contratar el servicio Mail Express o Mensaje Resumen |\n| |\n| Acceso Web |\n| |\n| En Internet , en la direcci\u00f3n www correo movistar es el cliente podr\u00e1 activar su cuenta de correo, configurar el servicio de suscripci\u00f3n y acceder a la bandeja de entrada de las cuentas de correo movistar Se solicitar\u00e1 introducir n? de m\u00f3vil y una clave de acceso, que es la misma clave utilizada para servicios como Mensajer\u00eda Multimedia, Mensajer\u00eda Web, Copiagenda, etc La clave se obtiene enviando un SMS al 22770 con la palabra CLAVE En el caso que sea la primera vez que el cliente entre en la web, le dar\u00e1 la opci\u00f3n de ?movilizar? una cuenta de correo, le indicar\u00e1 que introduzca los datos b\u00e1sicos de la cuenta y si necesita realizar un reenv\u00edo autom\u00e1tico lo indicar\u00e1 Posteriormente el cliente podr\u00e1 contratar mail express o mensaje resumen |\n| |\n| Acceso Vocal |\n| |\n| Se habilitan distintos n\u00fameros de tel\u00e9fono para acceder al Servicio vocal, el cliente deber\u00e1 llamar al 22321 desde un tel\u00e9fono Movistar o al 629000321 desde cualquier otro tel\u00e9fono, para contratar el servicio Tras una locuci\u00f3n se les enviar\u00e1 un wap push con el proceso de configuraci\u00f3n de la cuenta |\n| |\n| Experiencia de uso - Se detalla la experiencia de uso desde cada uno de los interfaces: Mail Express - El cliente recibe al instante un mensaje multimedia con el contenido del correo que acaba de recibir en su cuenta de correo de Hotmail, yahoo, gmail ? - Este mensaje incluye hasta 3 000 caracteres del mensaje original pero no incorpora ning\u00fan tipo de adjuntos - El cliente podr\u00e1 responder a este correo o remitirselo a quien quiera Sin embargo el precio de este mensaje no est\u00e1 incluido en la cuota del servicio El remitente de estos mensajes de respuesta ser\u00e1 la cuenta de correo que tenga movilizada ( ejemplo: pepe@terra es) Mensajes Resumen - Todos los d\u00edas el cliente recibir\u00e1 un mensaje con el resumen de los correos recibidos en su cuenta de correo en las \u00faltimas 24 horas Podr\u00e1 seleccionar el correo y acceder a emocion para ver el contenido del mensaje completo Dispositivos de acceso - El servicio est\u00e1 disponible para todos los terminales con acceso a emoci\u00f3n, aunque es deseable que tengan capacidades multimedia |\n| |\n| Requisitos T\u00e9cnicos e Incompatibilidades |\n| |\n| Compatible con todos los contratos abiertos comercialmente excepto Contrato Internet Plus y Contrato Internet |\n| |\n| C\u00f3mo se Dispone del Servicio |\n| |\n| Para activar el servicio el cliente se puede dar de alta a trav\u00e9s de los siguientes medios - A trav\u00e9s de www correomovil movistar es / www correo movistar es - A trav\u00e9s de emoci\u00f3n> correo/Chat> moviliza tu cuenta - Llamando a los siguientes n\u00fameros: - 22321: para altas en cuenta movistar - 223210: altas en cuenta Hotmail - 223211 alta del servicio para cualquier cuenta habilitada El cliente elige su cuenta - 223212: alta en cuenta de yahoo - 223213:alta en cuenta de terra - 223214:alta en cuenta de telefonica net Tras llamar a cualquiera de estos n\u00fameros, el cliente escuchar\u00e1 una locuci\u00f3n y si confirma recibe un SMS Push para acceder en navegaci\u00f3n m\u00f3vil y dar de alta el servicio |\n| |\n\n||\n| - |\n| M\u00e1s Informaci\u00f3n |\n| |\n| Mail Express: |\n| |\n| Direcci\u00f3n de correo del originante, en MMS 930, 931 y 943 en SMS / Wap Push Mensaje Resumen: 941 y 944 |\n| |\n| Cuentas de Correo |\n| |\n| Un usuario, con o sin cuenta de Movistar, puede asociar: - 1 cuenta Hotmail - 1 cuenta Yahoo - 1 cuenta Terra - 1 cuenta Telefonica net - hasta 5 cuentas m\u00e1s de otro proveedor |\n\n||\n| - |\n| Beneficios |\n| Universalidad Permite disponer del correo en cualquier tel\u00e9fono F\u00e1cil de provisi\u00f3n y de uso V\u00e1lido para los principales proveedores de correo del mercado Instant\u00e1neo: recibe tu correo al momento en tu m\u00f3vil |\nCodificaci\u00f3n ATOS\n===\n\n||\n| - |\n| Visualizaci\u00f3n y Codificaci\u00f3n en ATOS |\n| |\n| MS Empresas: Servicios / Transmisi\u00f3n de Datos / Correo M\u00f3vil Residencial: Servicios/Correo Electr\u00f3nico/Correo M\u00f3vil |\n| |\nCoste\n===\n\n||\n| - |\n| Coste |\n| Mail Express 2?/mes Mensajes Resumen: 1?/mes Mensajes de respuesta a email: precio regular del MMS Accesos a emoci\u00f3n Precios regulares de emoci\u00f3n Consulta de correo a trav\u00e9s de una llamada al 22321: 0,005 euros/seg Establecimiento de llamada: 0,15 euros Impuestos Indirectos No Incluidos |\nComplemento\n===\n\n||\n| - |\n| Periodo de Vigencia |\n| |\n| Este servicio se cerrar\u0102\u00a1 comercialmente el 31 de marzo de 2011 |\nComplemento\n===\n\n||\n| - |\n| Operativa |\n| |\n| MS Empresas: |\n| |\n| Pincha aqu\u00ed para ver Operativa Baja Pincha aqu\u00ed para ver Operativa de Incidencias Pincha aqu\u00ed para ver Operativa de Configuraci\u00f3n |\n| |\n| MS Plus/ Activa: |\n| |\n| Pincha aqu\u00ed para ver Operativa baja Pincha aqu\u00ed para ver Operativa incidencia Pincha aqu\u00ed para ver Operativa de Configuraci\u00f3n |\n
"""

qa_pairs = generate_answers_and_questions(markdown)

print(qa_pairs)

RESPONSE: [[
    {"question": "¿Qué es el servicio de correo móvil?", 
     "answer": "Es un servicio de suscripción que permite al cliente recibir mensajes en el móvil con información sobre los correos recibidos en las cuentas de correo que haya configurado."},
    {"question": "¿Qué vía de notificación se utiliza por defecto en el servicio de correo móvil?", 
     "answer": "La vía de notificación por defecto es MMS."},
    {"question": "¿Para qué cuentas de correo está disponible el servicio de Mensaje Mail Express?", 
     "answer": "Este servicio está disponible para las cuentas de Windows Live Hotmail, Yahoo, Gmail, Terra, Telefonica net y todas aquellas que tengan habilitado el acceso POP 3/Imap 4."},
    {"question": "¿Cuál es el precio del servicio de Mensaje resumen?", 
     "answer": "El precio de suscripción a este servicio es de 1 euro al mes, impuestos indirectos no incluidos."},
    {"question": "¿Qué información incluye el Mensaje resumen?", 
     "answer": "La informac

## Create the Excel file with question and answer pairs to evaluate the answers generated

In [10]:
import pandas as pd

input_dir = '../data_out/markdown_files'
markdown_contents = load_files(input_dir, '.txt')
data = {'question': [],
        'answer': []
}
for i, markdown_content in enumerate(markdown_contents):
    print(f"[{i + 1}]: title: {markdown_content['title']}")
    #print(f"\t content: [{markdown_content['content']}]")
    title = markdown_content['title'].replace('.txt', '')
    markdown = markdown_content['content']
    qa_pairs = generate_answers_and_questions(title + '. ' + markdown)
    for qa in qa_pairs:
        data['question'].append(qa['question'])
        data['answer'].append(qa['answer'])

# Save questions and answers pairs in an Excel file
df = pd.DataFrame(data)
output_dir = '../data_out'
os.makedirs(output_dir,exist_ok=True)
output_file = output_dir + '/qa_pairs.xlsx'
df.to_excel(output_file, index=False)
print(f'File {output_file} saved')


Loading files in markdown_files...
[1]: title: Activating a new customer account 2.txt
	[1]: question: [What is the purpose of the New Customer Account Activation Guide at QuickConnect?]
	 answer: [The guide provides a systematic approach for customer service representatives to facilitate the seamless activation of new customer accounts, ensuring a positive onboarding experience.]
	[2]: question: [What are the necessary requirements for activating a new customer account at QuickConnect?]
	 answer: [Customer's personal details, identification documents, service plan selection details, payment details, and access to the QuickConnect activation portal.]
	[3]: question: [What is the first step in activating a new customer account according to the guide?]
	 answer: [Gather necessary personal details from the customer, including their full name, address, date of birth, and contact number.]
	[4]: question: [How does the guide suggest handling the payment process during account activation?]
	 